In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from functools import reduce


import pandas as pd
from multiprocessing import Pool
from nltk.corpus import stopwords
from nltk.corpus import stopwords
import numpy as np
stopWords = []
for i in """!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'""":
    stopWords.append(i)
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from operator import itemgetter
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from typing import List, Dict
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import scale

import numpy as np
np.random.seed(42)
import pandas as pd
from multiprocessing import Pool

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '32'


def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
            
        else:
            yield word

from nltk.corpus import stopwords
stopWords = []
for i in """!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'""":
    stopWords.append(i)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()
lemma_dict = {}
def lemmatizer(word):
    try:
        word = word.lower()
        if word in lemma_dict:
            return lemma_dict[word]
        else:
            normal_form = ps.stem(word)
            lemma_dict[word] = normal_form
            return normal_form
    except:
        return 'nonascii'
def join_2(x,*args):
    if len(args) == 0:
        return x
    else:
        return x+" " +args[0]
def transform(x):
    try:
        for symbol in stopWords:
            if symbol in x:
                x = x.replace(symbol," ")
                x = x.replace('  '," ")
                x = x.replace('   '," ")
        x = x.split()
        x = map(lemmatizer,x)
        #x = map(asc,x)
        x =  reduce(join_2,x)
        return x
    except:
        return "problem"

/home/n01z3/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
y = train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
train.drop(['toxic','severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

merge = pd.concat((train,test))

ntrain = train.shape[0]
print(train.shape,test.shape,merge.shape)

def how_many_upper(text):
    upper = 0
    for char in text:
        if char.isupper():
            upper+=1
    return upper

def text_len(text):
    return len(text)

def count_of_stop_words(text):
    count = 0
    for char in text:
        if char in stopWords:
            count+=1
    return count

def clean_text(text):
    for symbol in stopWords:
        text = text.replace(symbol," ")
        text = text.lower()
        text = text.replace('  '," ")
        text = text.replace('   '," ")
    return text

pictures = ['.gif','.jpg','.png','.tiff']

def has_picture(text):
    has = 0
    for _ in pictures:
        if _ in text:
            has = 1
    return has

(159571, 2) (153164, 2) (312735, 2)


In [3]:
p = Pool(processes=32)
merge['upper_symbols'] = p.map(how_many_upper, merge.comment_text.values)
p.terminate()

p = Pool(processes=32)
merge['text_len'] = p.map(text_len, merge.comment_text.values)
p.terminate()

merge['percentage_of_upper'] = merge.upper_symbols/merge.text_len

p = Pool(processes=32)
merge['stop_words_count'] = p.map(count_of_stop_words, merge.comment_text.values)
p.terminate()

merge['percentage_of_stop'] = merge.stop_words_count/merge.text_len

p = Pool(processes=32)
merge['has_picture'] = p.map(has_picture, merge.comment_text.values)
p.terminate()

p = Pool(processes=32)
merge.comment_text = p.map(transform, merge.comment_text.values)
p.terminate()

def split(text):
    return len(text.split())

p = Pool(processes=32)
merge['number_of_words'] = p.map(split, merge.comment_text.values)
p.terminate()

words_95 = np.percentile(merge.number_of_words,95)


merge.text_len = scale(merge.text_len)
merge.upper_symbols = scale(merge.upper_symbols)
merge.stop_words_count = scale(merge.stop_words_count)
merge.number_of_words = scale(merge.number_of_words)

In [4]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [5]:
folds = list(range(len(train)))
folds = np.array(folds) % 4

In [6]:
def lemma(text):
    return " ".join(lemmatize_all(str(text)))
train_holder = train['comment_text'].copy() 
test_holder = test['comment_text'].copy() 
from multiprocessing import Pool


In [7]:

X_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values

X_train1=[]
X_test1=[]
#Function call to lemmatize X_train and X_test
print ("Train data lemmatization begins")

    
p = Pool(processes=32)
train_holder = p.map(lemma, train.comment_text.values)
p.terminate()
X_train1 = list(train_holder)
    
print ("Train data lemmatization ends")
print ("Test data lemmatization begins")

    
p = Pool(processes=32)
test_holder = p.map(lemma, test.comment_text.values)
p.terminate()
X_test1 = list(test_holder)    

print ("Test data lemmatization ends")

max_features = 80000
maxlen = 200
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train1) + list(X_test1))
X_train = tokenizer.texts_to_sequences(X_train1)
X_test = tokenizer.texts_to_sequences(X_test1)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)


def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))



Train data lemmatization begins
Train data lemmatization ends
Test data lemmatization begins
Test data lemmatization ends


In [8]:
x_train = x_train.astype('str').tolist()
x_test = x_test.astype('str').tolist()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

tfidf = TfidfVectorizer(analyzer=lambda x: x)
tfidf.fit(x_train + x_test)
dictionary = embeddings_index
max_idf = max(tfidf.idf_)
word2weight = defaultdict(lambda: max_idf,[(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
def transform(X): # усрденям слова в предложении
    
    dim = len(next(iter(dictionary.values())))
    return np.mean(np.array([
        np.mean([dictionary[w]*word2weight[w] for w in words if w in dictionary] 
                or [np.zeros(dim)], axis=0)
        for words in X
    ]),axis=0)


In [10]:

p = Pool(processes=12)
x_train = p.map(transform, x_train )
p.terminate()

In [11]:

p = Pool(processes=12)
x_test = p.map(transform, x_test )
p.terminate()

In [12]:
from keras.layers import Input, Dropout, Dense, BatchNormalization,MaxPooling1D, \
    Activation, concatenate, GRU, Embedding, Flatten,LSTM,Reshape
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping#, TensorBoard
from keras import backend as K
from keras import optimizers
from keras import initializers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import Conv1D
from keras.layers.advanced_activations import ELU


In [13]:
def get_model():
    inp = Input(shape=(x_train[0].shape[0]+7, ))
    
    

    
    conc = Dense(1500,activation='relu')(inp)
    conc = Dense(1500,activation='relu')(conc)
    conc = Dense(300)(conc)
    conc = Dense(1500,activation='relu')(conc)
    conc = Dense(1500,activation='relu')(conc)
    
    outp = Dense(x_train[0].shape[0]+7)(conc)
    
    model = Model(inputs=inp, 
        outputs=outp)
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['mse'])

    return model



In [14]:
def make_dict_predict(df_1,df_2):
    dict_X_train = pd.concat((df_1,df_2),axis=1)
    return dict_X_train



In [15]:
dict_OOF_predict = {}
dict_y_predict = {}

for fold in range(1):
    model = get_model()
    
    callbacks = [EarlyStopping(monitor='val_loss',
                               patience=4,
                               verbose=1,
                               min_delta=1e-4),
                 ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.1,
                                   patience=2,
                                   cooldown=2,
                                   verbose=1),
                 ModelCheckpoint(filepath="autoencoder.hdf5",
                                 save_best_only=True,
                                 save_weights_only=True,verbose=1,monitor='val_loss')]



    dict_all = make_dict_predict(pd.concat((pd.DataFrame(x_train),pd.DataFrame(x_test))),
                                merge[['upper_symbols','text_len','percentage_of_upper',
       'stop_words_count','percentage_of_stop','has_picture','number_of_words']])
    
    

    
    model.fit(dict_all,dict_all,epochs=2000,validation_data=(dict_all,dict_all),
            callbacks=callbacks,batch_size=256)


    print('fold {} is finished'.format(fold))
    

Train on 312735 samples, validate on 312735 samples
Epoch 1/2000
312320/312735 [============================>.] - ETA: 0s - loss: 0.0027 - mean_squared_error: 0.0027
Epoch 00001: val_loss improved from inf to 0.00208, saving model to autoencoder.hdf5
312735/312735 [==============================] - 64s 205us/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.0021 - val_mean_squared_error: 0.0021
Epoch 2/2000
312576/312735 [============================>.] - ETA: 0s - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 00002: val_loss did not improve
312735/312735 [==============================] - 61s 196us/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 3/2000
312320/312735 [============================>.] - ETA: 0s - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 00003: val_loss did not improve
312735/312735 [==============================] - 61s 194us/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.002

In [16]:
def get_model_2():
    inp = Input(shape=(x_train[0].shape[0]+7, ))
    
    

    
    conc = Dense(1500,activation='relu')(inp)
    conc = Dense(1500,activation='relu')(conc)
    conc = Dense(300)(conc)
    conc = Dense(1500,activation='relu')(conc)
    conc = Dense(1500,activation='relu')(conc)
    
    model = Model(inputs=inp, 
        outputs=conc)
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['mse'])

    return model
model_2 = get_model_2()
model_2.set_weights(model.get_weights()[0:len(model_2.weights)])
X_last = model_2.predict(dict_all)

In [17]:
def get_model_2():
    inp = Input(shape=(x_train[0].shape[0]+7, ))
    
    

    
    conc = Dense(1500,activation='relu')(inp)
    conc = Dense(1500,activation='relu')(conc)
    conc = Dense(300)(conc)
    conc = Dense(1500,activation='relu')(conc)

    
    model = Model(inputs=inp, 
        outputs=conc)
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['mse'])

    return model
model_2 = get_model_2()
model_2.set_weights(model.get_weights()[0:len(model_2.weights)])
X_last_2 = model_2.predict(dict_all)

In [18]:
def get_model_2():
    inp = Input(shape=(x_train[0].shape[0]+7, ))
    
    

    
    conc = Dense(1500,activation='relu')(inp)
    conc = Dense(1500,activation='relu')(conc)
    conc = Dense(300)(conc)

    
    model = Model(inputs=inp, 
        outputs=conc)
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['mse'])

    return model
model_2 = get_model_2()
model_2.set_weights(model.get_weights()[0:len(model_2.weights)])
X_last_3 = model_2.predict(dict_all)

In [ ]:
def get_model_2():
    inp = Input(shape=(x_train[0].shape[0]+7, ))
    
    

    
    conc = Dense(1500,activation='relu')(inp)
    conc = Dense(1500,activation='relu')(conc)


    
    model = Model(inputs=inp, 
        outputs=conc)
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['mse'])

    return model
model_2 = get_model_2()
model_2.set_weights(model.get_weights()[0:len(model_2.weights)])
X_last_4 = model_2.predict(dict_all)

In [ ]:
def get_model_2():
    inp = Input(shape=(x_train[0].shape[0]+7, ))
    
    

    
    conc = Dense(1500,activation='relu')(inp)


    
    model = Model(inputs=inp, 
        outputs=conc)
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['mse'])

    return model
model_2 = get_model_2()
model_2.set_weights(model.get_weights()[0:len(model_2.weights)])
X_last_5 = model_2.predict(dict_all)

In [ ]:
f_0,f_1,f_2,f_3 = dict_y_predict.keys()
sub = dict_y_predict[f_0] + dict_y_predict[f_1] + dict_y_predict[f_2] + dict_y_predict[f_3]
sub/= 4.0

f_0 = 'fold_0'
f_1 = 'fold_1'
f_2 = 'fold_2'
f_3 = 'fold_3'
OOF_X = np.zeros_like(y_train)
OOF_X = pd.DataFrame(OOF_X)

OOF_X[folds==0] = dict_OOF_predict[f_0]
OOF_X[folds==1] = dict_OOF_predict[f_1]
OOF_X[folds==2] = dict_OOF_predict[f_2]
OOF_X[folds==3] = dict_OOF_predict[f_3]

from sklearn.metrics import roc_auc_score,accuracy_score
print(roc_auc_score(np.array(y_train),(np.array(OOF_X))))

In [ ]:
submit = pd.read_csv('sample_submission.csv')
submit[['toxic','severe_toxic','obscene','threat','insult','identity_hate']] = sub

pd.DataFrame(OOF_X).to_csv('../second_level/OOF_conv_feat_2.csv',index=None)
submit.to_csv('../second_level/conv_feat_2.csv',index=None)